In [24]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [36]:
name = 'bank'
# df = pd.read_csv('/media/6TB_Volume/DataRepo/small_datasets/defaultCredit/data.csv')
df = pd.read_csv('/media/6TB_Volume/DataRepo/small_datasets/{}/data.csv'.format(name))
# target_clm = 'default.payment.next.month'
target_clm = 'y'
#get numerical columns
dtyp_df = pd.read_csv('/media/6TB_Volume/DataRepo/small_datasets/{}/fold0/data_types.csv'.format(name))
num_types = dtyp_df[dtyp_df.type.isin(['pos','count', 'real'])].index
num_types = pd.DataFrame(columns=list(df.columns[num_types]))
cat_types = dtyp_df[~dtyp_df.type.isin(['pos','count', 'real'])].index
num_categories = dtyp_df[~dtyp_df.type.isin(['pos','count', 'real'])].dim
cat_types = df.columns[cat_types]
cat_types = pd.DataFrame(columns=cat_types)
cat_types.loc[0] = num_categories.values
cat_types = cat_types.drop([target_clm], axis=1)
# factorize the categorical columns
df[cat_types.columns] = df[cat_types.columns].apply(lambda x: pd.factorize(x)[0])
#scale numerical columns
scaler = StandardScaler()
# first we need to detect numerical clms and then scale, this is incorrect
df[num_types.columns] = scaler.fit_transform(df[num_types.columns])
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,1.533034,0,0,0,0,0,0,0,0,0,...,-0.565922,0.195414,-0.349494,0,0.648092,0.722722,0.886447,0.71246,0.33168,0
1,1.628993,1,0,1,1,0,0,0,0,0,...,-0.565922,0.195414,-0.349494,0,0.648092,0.722722,0.886447,0.71246,0.33168,0
2,-0.290186,1,0,1,0,1,0,0,0,0,...,-0.565922,0.195414,-0.349494,0,0.648092,0.722722,0.886447,0.71246,0.33168,0
3,-0.002309,2,0,2,0,0,0,0,0,0,...,-0.565922,0.195414,-0.349494,0,0.648092,0.722722,0.886447,0.71246,0.33168,0
4,1.533034,1,0,1,0,0,1,0,0,0,...,-0.565922,0.195414,-0.349494,0,0.648092,0.722722,0.886447,0.71246,0.33168,0


In [38]:
num_types.to_csv('/home/vineeth/Documents/GitWorkSpace/PytorchRecipes/SimpleMLP/Dataset/{}/numerical_clms.csv'.format(name), \
     index=False)
cat_types.to_csv('/home/vineeth/Documents/GitWorkSpace/PytorchRecipes/SimpleMLP/Dataset/{}/categorical_clms.csv'.format(name), \
     index=False)

In [40]:
df_train, df_test = train_test_split(df, test_size=0.2, random_state=142)
df_train, df_valid = train_test_split(df_train, test_size=0.1, random_state=142)
df_train.to_csv('/home/vineeth/Documents/GitWorkSpace/PytorchRecipes/SimpleMLP/Dataset/{}/fold0/train/data.csv'.format(name), index=False)
df_valid.to_csv('/home/vineeth/Documents/GitWorkSpace/PytorchRecipes/SimpleMLP/Dataset/{}/fold0/valid/data.csv'.format(name), index=False)
df_test.to_csv('/home/vineeth/Documents/GitWorkSpace/PytorchRecipes/SimpleMLP/Dataset/{}/fold0/test/data.csv'.format(name), index=False)

In [103]:
cat_types[len(cat_types.index)] = num_categories
# num_categories = num_categories.to_frame(index=False).T

In [114]:
num_categories = num_categories.to_frame().T

In [115]:
num_categories

,1,2,3,5,6,7,8,9,10,23
dim,3,7,4,11,11,11,11,11,11,2
